In [17]:
!pip install opencv-python deepface numpy mtcnn scipy

In [18]:
import cv2
import os

def extract_frames(video_path, output_folder, frame_interval=30):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    
    frame_count = 0
    success, frame = cap.read()
    
    while success:
        if frame_count % frame_interval == 0:  # Extract every Nth frame
            frame_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)
        success, frame = cap.read()
        frame_count += 1

    cap.release()
    return output_folder

In [19]:
!pip install tf-keras
from deepface import DeepFace
import numpy as np

def get_face_embedding(face_img):
    try:
        embedding = DeepFace.represent(face_img, model_name="ArcFace")[0]["embedding"]
        return np.array(embedding)
    except Exception as e:
        print(f"Embedding generation failed: {e}")
        return None

In [20]:
from scipy.spatial.distance import cosine

def is_same_person(embedding1, embedding2, threshold=0.5):
    if embedding1 is None or embedding2 is None:
        return False
    return cosine(embedding1, embedding2) < threshold

In [21]:
from mtcnn import MTCNN

detector = MTCNN()

def detect_faces(image_path):
    image = cv2.imread(image_path)
    faces = detector.detect_faces(image)
    
    if faces:
        x, y, width, height = faces[0]['box']
        face_img = image[y:y+height, x:x+width]
        return face_img
    return None

In [22]:
def process_videos(video_paths):
    reference_embedding = None  # Only set once from the first video
    results = {}

    for idx, video_path in enumerate(video_paths):
        print(f"Processing {video_path}...")

        # Step 1: Extract frames
        output_folder = f"frames_video_{idx}"
        extract_frames(video_path, output_folder)

        # Step 2: Detect face in the first extracted frame
        frame_files = sorted(os.listdir(output_folder))
        detected_face = None

        for frame in frame_files:
            face_img = detect_faces(os.path.join(output_folder, frame))
            if face_img is not None:
                detected_face = face_img
                break

        if detected_face is None:
            results[video_path] = "No face detected"
            continue

        # Step 3: Extract face embedding
        embedding = get_face_embedding(detected_face)

        if embedding is None:
            results[video_path] = "Face detected, but embedding not generated"
            continue

        # Step 4: Compare with reference face
        if reference_embedding is None:
            reference_embedding = embedding  # Set only for the first video
            results[video_path] = "Reference face set"
        else:
            if is_same_person(reference_embedding, embedding):
                results[video_path] = "Same face detected"
            else:
                results[video_path] = "Different face detected"

    return results

In [23]:
video_list = ["Face Recognition/Test1.mov", "Face Recognition/Test2.mov", "Face Recognition/Test3.mov"]
verification_results = process_videos(video_list)

for video, result in verification_results.items():
    print(f"{video}: {result}")

Processing Face Recognition/Test1.mov...


Embedding generation failed: module 'deepface.modules.modeling' has no attribute 'build_model'
Processing Face Recognition/Test2.mov...
Embedding generation failed: module 'deepface.modules.modeling' has no attribute 'build_model'
Processing Face Recognition/Test3.mov...
Embedding generation failed: module 'deepface.modules.modeling' has no attribute 'build_model'
Face Recognition/Test1.mov: Face detected, but embedding not generated
Face Recognition/Test2.mov: Face detected, but embedding not generated
Face Recognition/Test3.mov: Face detected, but embedding not generated
